In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Scrape News Title and Web Links

- The Star Online: ???
- MalaysiaKini: 

| News | Link | Period |
| --- | --- | --- |
| The Malay Mail | https://www.malaymail.com/archives | 2013/12 - 2021/02|

- Backup 
    - The Sun: https://www.thesundaily.my/archive

In [1]:
url = 'https://www.malaymail.com/news/malaysia/2013?page=1'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')

NameError: name 'requests' is not defined

In [29]:
titles_links = []

for a in soup.find_all('a', class_='py-2 d-block'):
    titles_links.append([a.text, a['href']])

In [37]:
titles_links[:3]

[['SPAD yet to approve new fare and surcharge on school bus',
  'https://www.malaymail.com/news/malaysia/2013/12/31/spad-yet-to-approve-new-fare-and-surcharge-on-school-bus/590609'],
 ['Government to monitor effect of rising cost of living, says Muhyiddin',
  'https://www.malaymail.com/news/malaysia/2013/12/31/government-to-monitor-effect-of-rising-cost-of-living-says-muhyiddin/590607'],
 ['Government yet to issue directive on toll rate hike',
  'https://www.malaymail.com/news/malaysia/2013/12/31/government-yet-to-issue-directive-on-toll-rate-hike/590603']]

In [32]:
len(titles_links)

50

- What's the number of news per month?
    - 2018/01 The Malay Mail 
    - 32 pages$*$50 news per page = 1600 news
    - So news for a year will be 1600$*$12 = 19200 news


# Scrape News Articles

In [33]:
url = 'https://www.malaymail.com/news/malaysia/2013/12/31/spad-yet-to-approve-new-fare-and-surcharge-on-school-bus/590609'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')

In [48]:
for p in soup.find_all('article')[0].find_all('p'):
    print(p.text)



Subscribe to our Telegram channel for the latest updates on news you need to know.
KUALA LUMPUR, Dec 31 — The Land Public Transport Commission (SPAD) has not approved any new fare and surcharge on school buses as quoted in certain news dailies recently.
The commission, in its statement today, explained that consent had been reached with representatives from The School Bus Associations of Malaysia (PPPBSM) and the Malaysian Federation of School Bus Operators (GPBSM) that there would be no surchange or price hikes in school bus fares until SPAD announced the new school bus fare scheme in March 2014. 
“Currently SPAD is finalising the new school bus fare scheme while taking into account three significant elements.
“Therefore, all operators should maintain the current school bus fare structure as approved by the Commercial Vehicle Licensing Board in August 2009,” the statement said.
According to the statement, the three elements included conducting engagement sessions with parents/ schoo

# Malay Mail

| Section | Link | Period | Pages | Remark | 
| -- | -- | -- | -- | -- |
| Opinion | [link](https://www.malaymail.com/news/opinion/2013/06) | 2013-06-30 - Present | ?? | Mixed languages | 

- Last update: 20 June 2021

In [2]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait

import time

In [3]:
# Static version 
from bs4 import BeautifulSoup
import requests

# Scrape titles and links 
url = 'https://www.malaymail.com/news/opinion/2013/06'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')

titles_links = []

for a in soup.find_all('a', class_='py-2 d-block'):
    titles_links.append([a.text, a['href']])
    
# Scrape article's text 
url = 'https://www.malaymail.com/news/opinion/2013/06/30/syabas-zahid-kpn-baru/488729'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')

for p in soup.find_all('article')[0].find_all('p'):
    print(p.text)

## Scrape titles and links

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://www.malaymail.com/news/malaysia/2017')
# set wait until the page loaded

In [4]:
# Click the consent button for collection of personal data
# driver.find_elements_by_class_name("fc-button fc-cta-consent fc-primary-button")
# selenium seems not able to find button tag with class, use xpath instead 
# https://sqa.stackexchange.com/questions/45303/find-button-class-with-selenium-on-python 

consent_button = driver.find_element_by_xpath("//button[@class='fc-button fc-cta-consent fc-primary-button']")
ActionChains(driver).click(consent_button).perform()

In [5]:
# Get available years and months 
year_menu = Select(driver.find_element_by_id('inputYear'))
options = year_menu.options # get all available options 
years_options = [option.text for option in options]
print(years_options[0:5])

month_menu = Select(driver.find_element_by_id('inputMonth'))
options = month_menu.options # get all available options 
month_options = [option.text for option in options]
month_options.remove('All')
print(month_options[0:5])
# Note that 'all' is an option for month

['2013', '2014', '2015', '2016', '2017']
['January', 'February', 'March', 'April', 'May']


In [8]:
# To check if an element exists
# https://stackoverflow.com/questions/9567069/checking-if-element-exists-with-python-selenium
        
def check_exists_by_xpath(xpath, timeout=15):
    '''Return a boolean that indicates the presence of an element given a xpath.'''
    try:
        WebDriverWait(driver, timeout=timeout).until(
            EC.presence_of_element_located((By.XPATH, xpath)))
    except:
        return False
    return True

In [9]:
def scrape_titles_links(year_month):
    # Collect titles and urls of articles
    titles_links = []
    
    try:
        articles = WebDriverWait(driver, timeout=10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//a[@class='py-2 d-block']"))
        ) # make sure all elements are loaded

        for a in articles:
            title = a.text
            link = a.get_attribute('href')
            titles_links.append([year_month, title, link])
    
    except: 
        # Pass if no articles were found
        if driver.find_element_by_xpath("//div[@class='alert alert-warning']/p").text == 'No articles found.':
            print('No articles found.')

    return titles_links 

In [12]:
year = '2017'

# Iterate through all months to get news
titles_links = []
for index, month in enumerate(month_options):
    # Identify drop down menu for months 
    month_menu = Select(driver.find_element_by_id('inputMonth')) 

    # Select month by text if no duplicates present
    if len(month_options) == len(set(month_options)): 
        month_menu.select_by_visible_text(month)
    else:
    # Select month by index if duplicates present
        month_menu.select_by_index(index)

    # Click submit button 
    driver.find_element_by_xpath("//button[@type='submit']").click() 

    next_page = True 
    while next_page:
        # If next button is available 
        if check_exists_by_xpath("//a[@class='page-link'][@rel='next']"):
            # Collect titles and urls of articles on present page
            titles_links.extend(scrape_titles_links(year + '_' + month))

            # Click the next button when it's clickable
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[@class='page-link'][@rel='next']")))
            action = ActionChains(driver)
            action.click(next_button) 
            action.perform()
            
            # Navigate to the advertisement iframe if present
            if check_exists_by_xpath("//iframe[@id='aswift_1']"):
                print('advertisement identified.')
                frame = driver.find_element_by_xpath("//iframe[@id='aswift_1']")
                driver.switch_to.frame(frame)
                print('driver switched to iframe.')

                if check_exists_by_xpath("//div/div[@id='dismiss-button']", 3):
                    # Ad type 1: close button on top right
                    driver.find_element_by_xpath(("//div/div[@id='dismiss-button']")).click()
                    print('advertisement closed.')
                elif check_exists_by_xpath("//iframe[@id='ad_iframe']"):
                    # Ad type 1: close button within the ad 
                    # Switch to iframe inside iframe
                    frame = driver.find_element_by_xpath("//iframe[@id='ad_iframe']")
                    driver.switch_to.frame(frame)
                    print('done step2')

                    # Click the close button 
                    try:
                        WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.XPATH, "//div/div[@id='dismiss-button']"))).click()
                        #driver.find_element_by_xpath(("//div/div[@id='dismiss-button']")).click()
                        print('done step3')
                    except:
                        pass
                else:
                    pass
        else:
            # Stop when there 
            print(month, 'last page.')

            # Collect titles and urls of articles
            titles_links.extend(scrape_titles_links(year + '_' + month))
            print(month, 'No. of articles:', len(titles_links))

            next_page = False

advertisement
done step1
ad 1 done


KeyboardInterrupt: 

In [14]:
len(titles_links)

536

In [19]:
df = pd.DataFrame(titles_links)
df.columns = ['year_month', 'title', 'link']
df

,year_month,title,link
0,2013_June,"Syabas Zahid, KPN baru",https://www.malaymail.com/news/opinion/2013/06...
1,2013_June,Nusrat Fateh dan konsep muzik yang pelbagai,https://www.malaymail.com/news/opinion/2013/06...
2,2013_June,Expect thriller from Brazil and Spain,https://www.malaymail.com/news/opinion/2013/06...
3,2013_June,Man of Steel: Too much steel?,https://www.malaymail.com/news/opinion/2013/06...
4,2013_June,Paris’ gay community post-same-sex marriage law,https://www.malaymail.com/news/opinion/2013/06...
...,...,...,...
531,2013_December,Winner-takes-all Bangladesh looks for way forw...,https://www.malaymail.com/news/opinion/2013/12...
532,2013_December,Fight for your right to party!,https://www.malaymail.com/news/opinion/2013/12...
533,2013_December,You’ve got to be joking,https://www.malaymail.com/news/opinion/2013/12...
534,2013_December,"After initial promise, Japan’s new economy ris...",https://www.malaymail.com/news/opinion/2013/12...


In [24]:
df.to_csv('data/malay_mail/opinion_' + year + '.csv', 
          index=False, encoding='utf-8-sig')

# https://stackoverflow.com/questions/25788037/pandas-df-to-csvfile-csv-encode-utf-8-still-gives-trash-characters-for-min/43684587

## Scrape text

In [10]:
driver = webdriver.Chrome()

In [11]:
driver.get('https://www.malaymail.com/news/malaysia/2019/01/31/mcmc-mulls-android-tv-box-ban-move-could-benefit-astro/1718651')

# Click the consent button for collection of personal data
consent_button = driver.find_element_by_xpath("//button[@class='fc-button fc-cta-consent fc-primary-button']")
ActionChains(driver).click(consent_button).perform()
time.sleep(2)

' '.join([text.text for text in driver.find_elements_by_xpath('//article/p')])

' KUALA LUMPUR, Jan 31 — Android TV boxes could be illegal in Malaysia as MCMC considers a move to ban the device. This, according to the commission, is on the premise to curb access to pirated content but it admits that the effort will need to involve other ministries and agencies. In addition, MCMC is prioritising its efforts on ways to prevent the download and streaming of pirated content rather than the out-and-out band of Android TV boxes. On the flipside, the ban could benefit Malaysia’s largest Pay-TV operator according to Maybank Investment Bank Research. The firm noted that Android TV boxes, which are gaining popularity in Malaysia, pose a huge problem for Pay-TV operators. “If Malaysia passes similar legislation, we believe that it would be a huge catalyst for Astro,” the firm told The Star when commenting on a similar ban being considered by the Singapore government. Singapore’s Law Ministry is looking to pass a law banning the sale of set-top boxes or media streaming boxes 

In [7]:
# Click the consent button for collection of personal data
consent_button = driver.find_element_by_xpath("//button[@class='fc-button fc-cta-consent fc-primary-button']")
ActionChains(driver).click(consent_button).perform()
time.sleep(2)

In [14]:
# Collect author and timestamp 
author_time = [text.text for text in driver.find_elements_by_xpath('//div[@class="byline"]/p')]

['Wednesday, 31 Jul 2013 07:13 AM MYT', 'By Erna Mahyuni']

In [8]:
' '.join([text.text for text in driver.find_elements_by_xpath('//article/p')])

' KUALA LUMPUR, Jan 31 — Android TV boxes could be illegal in Malaysia as MCMC considers a move to ban the device. This, according to the commission, is on the premise to curb access to pirated content but it admits that the effort will need to involve other ministries and agencies. In addition, MCMC is prioritising its efforts on ways to prevent the download and streaming of pirated content rather than the out-and-out band of Android TV boxes. On the flipside, the ban could benefit Malaysia’s largest Pay-TV operator according to Maybank Investment Bank Research. The firm noted that Android TV boxes, which are gaining popularity in Malaysia, pose a huge problem for Pay-TV operators. “If Malaysia passes similar legislation, we believe that it would be a huge catalyst for Astro,” the firm told The Star when commenting on a similar ban being considered by the Singapore government. Singapore’s Law Ministry is looking to pass a law banning the sale of set-top boxes or media streaming boxes 

In [19]:
pd.read_csv('data/malay_mail/opinion_2013.csv')

,year_month,title,link
0,2013_June,"Syabas Zahid, KPN baru",https://www.malaymail.com/news/opinion/2013/06...
1,2013_June,Nusrat Fateh dan konsep muzik yang pelbagai,https://www.malaymail.com/news/opinion/2013/06...
2,2013_June,Expect thriller from Brazil and Spain,https://www.malaymail.com/news/opinion/2013/06...
3,2013_June,Man of Steel: Too much steel?,https://www.malaymail.com/news/opinion/2013/06...
4,2013_June,Paris’ gay community post-same-sex marriage law,https://www.malaymail.com/news/opinion/2013/06...
...,...,...,...
531,2013_December,Winner-takes-all Bangladesh looks for way forw...,https://www.malaymail.com/news/opinion/2013/12...
532,2013_December,Fight for your right to party!,https://www.malaymail.com/news/opinion/2013/12...
533,2013_December,You’ve got to be joking,https://www.malaymail.com/news/opinion/2013/12...
534,2013_December,"After initial promise, Japan’s new economy ris...",https://www.malaymail.com/news/opinion/2013/12...


## Validation of texts

In [2]:
df = pd.read_csv('data/malay_mail/opinion_2013_complete.csv')
df.head()

,year_month,title,link,author,article
0,2013_June,"Syabas Zahid, KPN baru",https://www.malaymail.com/news/opinion/2013/06...,By Subky Abdul Latif,30 JUN — Syukur. Tahniah buat Blackout (Gelap)...
1,2013_June,Nusrat Fateh dan konsep muzik yang pelbagai,https://www.malaymail.com/news/opinion/2013/06...,By Meor Yusof Aziddin,30 JUN — Jika ditanya kepada saya siapa antara...
2,2013_June,Expect thriller from Brazil and Spain,https://www.malaymail.com/news/opinion/2013/06...,By Andy West,JUNE 29 — The Confederations Cup final between...
3,2013_June,Man of Steel: Too much steel?,https://www.malaymail.com/news/opinion/2013/06...,By Aidil Rusli,JUNE 29 — We tend to be a little bit possessiv...
4,2013_June,Paris’ gay community post-same-sex marriage law,https://www.malaymail.com/news/opinion/2013/06...,By Helen Hickey,JUNE 28 — How do they feel now? This is the qu...


In [4]:
df.isna().sum()

year_month    0
title         0
link          0
author        0
article       0
dtype: int64